<a href="https://colab.research.google.com/github/KrishnaSChavan/MachineLearning/blob/main/Clav2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google. colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"krishnaschavan","key":"6f1c6fc054fb8d6259f3b6411ea6191e"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed

100% 6.40G/6.40G [03:51<00:00, 26.4MB/s]
100% 6.40G/6.40G [03:51<00:00, 29.7MB/s]


In [5]:
import zipfile
zip_r = zipfile.ZipFile('/content/cervical-cancer-largest-dataset-sipakmed.zip')
zip_r.extractall()
zip_r.close()

In [6]:
!mkdir Data

In [7]:
one = "/content/im_Dyskeratotic/im_Dyskeratotic/"
two = "/content/im_Koilocytotic/im_Koilocytotic/"
three = "/content/im_Metaplastic/im_Metaplastic/"
four = "/content/im_Parabasal/im_Parabasal/CROPPED/"
five = "/content/im_Superficial-Intermediate/im_Superficial-Intermediate/"

In [8]:
path = [one,two,three,four,five]

In [9]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
data = []
labels = []
height = 64
width = 64
channels = 3
classes = 43
n_inputs = height * width * channels

i = 0
for p in path:
    Class=os.listdir(p)
    for a in Class:
        if(a[-1] == 'p'):
            labels.append(i)
            try:
                print(p+a)
                image=cv2.imread(p+a)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((height, width))
                data.append(np.array(size_image))
            except:
                print('')
    i+=1

In [11]:
len(data),len(labels)

(1645, 1645)

In [12]:
labels = np.array(labels)
data = np.array(data)

In [13]:
data.shape

(1645, 64, 64, 3)

In [14]:
labels

array([0, 0, 0, ..., 4, 4, 4])

In [15]:
categories = ["Dyskeratotic", "Koilocytotic", "Metaplastic", "Parabasal", "Superficial-Intermediate"]

In [16]:
def img(i):
    plt.imshow(data[i])
    plt.xlabel(categories[labels[i]])

In [17]:
np.random.seed(42)
s = np.arange(data.shape[0])
np.random.shuffle(s)
data = data[s]
labels = labels[s]

In [18]:
from keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam

In [19]:
resnet = ResNet50(weights='imagenet',include_top = False)

94765736/94765736 [==============================] - 3s 0us/step


In [20]:
IMAGE_SHAPE = (64,64)
def create_model(model_url, num_classes=5):

    feature_extraction_layer = hub.KerasLayer(model_url, trainable=False, name="feature_extraction", input_shape=IMAGE_SHAPE+(3,))

    # create our own model
    model = Sequential()
    model.add(feature_extraction_layer)
    model.add(Dense(num_classes, activation='softmax', name='output'))

    return model

In [21]:
scaled = data/255.

In [22]:
scaled.shape

(1645, 64, 64, 3)

In [23]:
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'

In [24]:
resnet_model = create_model(resnet_url)

In [25]:
# resnet_model.compile(
#     loss = "categorical_crossentropy",
#     optimizer = Adam(),
#     metrics=['accuracy']
# )
resnet_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Make sure this loss is appropriate for your task
    metrics=['accuracy'],
    run_eagerly=True  # Set to True to execute the model eagerly
)


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled, labels, test_size = 0.2, random_state = 1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1316, 64, 64, 3)
(329, 64, 64, 3)
(1316,)
(329,)


In [27]:
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
    log_dir = dir_name+'/'+experiment_name+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"saving tensorboard file to {log_dir}")
    return tensorboard_callback

In [28]:
from keras.utils import to_categorical

y_train_encoded = to_categorical(y_train, num_classes=5)
y_test_encoded = to_categorical(y_test, num_classes=5)
y = to_categorical(labels, num_classes=5)

In [33]:
# resnet_history=resnet_model.fit(
#             X_train,
#             y_test,
#             epochs = 10,
#             steps_per_epoch =len(X_train),
#             validation_data=X_test,
#             validation_steps=len(X_test),
#             callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',experiment_name='resnet50 v2')]
# )
resnet_history = resnet_model.fit(
    tdt,  # Assuming y_train contains the corresponding labels for X_train
    epochs=10,
)


Epoch 1/10
52/52 [==============================] - 21s 91ms/step - loss: 1.0113 - accuracy: 0.6286
Epoch 2/10
52/52 [==============================] - 4s 73ms/step - loss: 0.4853 - accuracy: 0.8213
Epoch 3/10
52/52 [==============================] - 3s 63ms/step - loss: 0.3857 - accuracy: 0.8742
Epoch 4/10
52/52 [==============================] - 3s 64ms/step - loss: 0.3251 - accuracy: 0.8948
Epoch 5/10
52/52 [==============================] - 4s 75ms/step - loss: 0.2801 - accuracy: 0.9167
Epoch 6/10
52/52 [==============================] - 3s 63ms/step - loss: 0.2498 - accuracy: 0.9264
Epoch 7/10
52/52 [==============================] - 4s 70ms/step - loss: 0.2211 - accuracy: 0.9368
Epoch 8/10
52/52 [==============================] - 4s 74ms/step - loss: 0.2021 - accuracy: 0.9422
Epoch 9/10
52/52 [==============================] - 3s 64ms/step - loss: 0.1830 - accuracy: 0.9502
Epoch 10/10
52/52 [==============================] - 5s 94ms/step - loss: 0.1668 - accuracy: 0.9556


In [ ]:
x_flattened = np.reshape(data, (data.shape[0], -1))

In [ ]:


X_train, X_test, Y_train, Y_test = train_test_split(x_flattened , labels, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (-1, data.shape[1], data.shape[2], data.shape[3]))
X_test = np.reshape(X_test, (-1, data.shape[1], data.shape[2], data.shape[3]))

In [ ]:
X_train.shape

(1316, 64, 64, 3)

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
from sklearn.utils import shuffle
IMAGE_SHAPE = (64,64)
BATCH_SIZE = 32
data_gen = ImageDataGenerator(rescale=(1/255.))

tdt = data_gen.flow(
    x = data,
    y=y,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# other stuff

In [ ]:
len(tdt)

52

In [ ]:
X_train.shape

(1316, 64, 64, 3)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(x_flattened)
principal_X = pca.transform(x_flattened)

In [ ]:
zzz = cv2.imread("/content/im_Dyskeratotic/im_Dyskeratotic/001.bmp")
image_from_array = Image.fromarray(zzz, 'RGB')
aaa = image_from_array.resize((height, width))

In [ ]:
aaa = np.array(aaa)

In [ ]:
aaa.shape

(64, 64, 3)

In [ ]:
aaa = aaa/255.

In [ ]:
reshaped_aaa = np.reshape(aaa, (-1, 64, 64, 3))


In [ ]:
asd = resnet_model.predict(reshaped_aaa)

1/1 [==============================] - 0s 26ms/step


In [ ]:
zzz = np.array(zzz)

In [ ]:
zzz.shape

()

In [ ]:
np.argmax(asd, axis=1)

array([0])

In [ ]:
resnet_model.save("CLAV")

In [ ]:
resnet_model.save("CLAV2",save_format="tf")

In [ ]:
resnet_model.save('CLAV3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loaded_model = tf.keras.models.load_model("CLAV3.h5")

In [ ]:
new_model = tf.keras.models.load_model('CLAV')

In [ ]:
import zipfile

# File or directory to be zipped
file_or_directory_to_zip = '/content/CLAV2'

# Name for the zip file
zip_filename = 'CLAVZ2.zip'

# Create a ZipFile object in write mode
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    if os.path.isfile(file_or_directory_to_zip):
        # If it's a single file, add it to the archive
        zipf.write(file_or_directory_to_zip, os.path.basename(file_or_directory_to_zip))
    elif os.path.isdir(file_or_directory_to_zip):
        # If it's a directory, add all files within the directory and its subdirectories
        for root, dirs, files in os.walk(file_or_directory_to_zip):
            for file in files:
                file_to_zip = os.path.join(root, file)
                archive_name = os.path.relpath(file_to_zip, file_or_directory_to_zip)
                zipf.write(file_to_zip, archive_name)

# The zip archive is created with the specified name


In [ ]:
import zipfile
zip_r = zipfile.ZipFile('/content/CLAVZ.zip')
zip_r.extractall()
zip_r.close()

In [ ]:
!mkdir CLAVZ

In [ ]:
!cd CLAVZ

In [ ]:
import zipfile
zip_r = zipfile.ZipFile('/content/CLAVZ.zip')
zip_r.extractall()
zip_r.close()

In [ ]:
new_model_1 = tf.keras.models.load_model('/content/CLAVZ')

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
zip_r = zipfile.ZipFile('/content/CLAVZ.zip')
zip_r.extractall()
zip_r.close()

# segmentation

In [34]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.cluster import KMeans

ValueError: ignored

In [37]:
X_train.shape

(1316, 64, 64, 3)

In [42]:
X_train[1].reshape((-1,3)).shape

(4096, 3)

In [41]:
len(X_train)

1316

In [63]:
cer = []
for xyz in X_train:
    i = xyz.reshape((-1,3))
    cer.append(np.array(i))

In [64]:
cer = np.array(cer)

In [65]:
cer.shape

(1316, 4096, 3)

In [66]:
cerv = []
for xyz in cer:
    aaa = xyz.reshape(-1)
    cerv.append(np.array(aaa))

In [67]:
cerv = np.array(cerv)

In [68]:
cerv.shape

(1316, 12288)

In [75]:
cer_test = []
for xyz in X_test:
    i = xyz.reshape((-1,3))
    cer_test.append(np.array(i))
cer_test = np.array(cer_test)

cerv_test = []
for xyz in cer_test:
    aaa = xyz.reshape(-1)
    cerv_test.append(np.array(aaa))

cerv_test = np.array(cerv_test)

In [76]:
cerv_test.shape

(329, 12288)

In [77]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(cerv, y_train)
y_svm = svm.predict(cerv_test)

In [78]:
y_svm.shape,y_test.shape

((329,), (329,))

In [79]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_svm)

In [80]:
acc

0.8145896656534954